In [1]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv('healthData.csv')

df = df.drop_duplicates(subset='ReferenceID')
df = df.dropna(subset=['DaysToReview', 'DaysToAdopt', 'DiagnosisCategory', 'PatientGender', 'AgeRange', 'Determination'])
df['DeterminationBinary'] = df['Determination'].map({
    'Overturned Decision of Health Plan': 0,
    'Upheld Decision of Health Plan': 1
})

X = pd.get_dummies(df[['DiagnosisCategory', 'PatientGender', 'AgeRange']], drop_first=True)
X['DaysToReview'] = df['DaysToReview']
X['DaysToAdopt'] = df['DaysToAdopt']
y = df['DeterminationBinary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

joblib.dump(rf_model, 'rf_flagging_model.pkl')

def flag_high_risk_case(new_data):
    model = joblib.load('rf_flagging_model.pkl')
    
    new_df = pd.DataFrame([new_data])
    new_df_encoded = pd.get_dummies(new_df)
    
    for col in X.columns:
        if col not in new_df_encoded:
            new_df_encoded[col] = 0
    new_df_encoded = new_df_encoded[X.columns]
    
    prediction = model.predict(new_df_encoded)
    
    if prediction[0] == 0:
        return "High Risk Flag"
    else:
        return "Low Risk"

def get_user_case():
    print("Enter details for the new patient case:")

    diagnosis = input("Enter Diagnosis Category (e.g., Mental Disorder, Autism Spectrum): ").strip()
    gender = input("Enter Patient Gender (Male or Female): ").strip()
    age_range = input("Enter Age Range (e.g., 0 to 10, 11 to 20, 41 to 50, 65+): ").strip()
    days_to_review = int(input("Enter Days to Review (e.g., 45): ").strip())
    days_to_adopt = int(input("Enter Days to Adopt (e.g., 30): ").strip())

    user_case = {
        f'DiagnosisCategory_{diagnosis}': 1,
        f'PatientGender_{gender}': 1,
        f'AgeRange_{age_range}': 1,
        'DaysToReview': days_to_review,
        'DaysToAdopt': days_to_adopt
    }
    
    return user_case

new_case = get_user_case()

flag_result = flag_high_risk_case(new_case)
print("\nPrediction for this case:", flag_result)

Enter details for the new patient case:
Enter Diagnosis Category (e.g., Mental Disorder, Autism Spectrum): Mental Disorder
Enter Patient Gender (Male or Female): Female
Enter Age Range (e.g., 0 to 10, 11 to 20, 41 to 50, 65+): 11 to 20
Enter Days to Review (e.g., 45): 60
Enter Days to Adopt (e.g., 30): 50

Prediction for this case: Low Risk
